# This notebook requires feature indices to be pasted from the mnist experiments.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset

import torchvision.datasets as visiondatasets
import torchvision.transforms as transforms

# Feature indices pasted from experiments

In [2]:
g1 = np.array([152, 211, 265, 319, 323, 324, 326, 354, 380, 404, 434, 489, 490, 576, 679])
g2 = np.array([93, 156, 260, 320, 349, 378, 382, 383, 402, 405, 489, 495, 569, 577, 659])
g3 = np.array([187, 216, 235, 238, 239, 241, 262, 272, 291, 297, 328, 353, 386, 469, 513])
g4 = np.array([210, 242, 268, 346, 347, 348, 350, 357, 405, 431, 436, 486, 512, 516, 543])

union = np.unique(np.concatenate([g1, g2, g3, g4]))


stg = np.array([152, 154, 155, 157, 185, 210, 212, 213, 243, 265, 268,
                 270, 272, 289, 291, 293, 294, 296, 316, 320, 324, 326,
                 328, 346, 349, 350, 371, 374, 378, 403, 405, 408, 409,
                 427, 434, 456, 465, 485, 486, 489, 514, 516, 540, 550,
                 571, 656])

scae = np.array([153, 657, 380, 578, 329, 317, 402, 538, 625, 457,
                 376, 456, 569, 440, 269, 455, 211,  99, 378, 554,
                 344, 343, 515, 348, 321, 374, 326, 292, 323, 433,
                 342, 552, 294, 486, 462, 157, 319, 387, 356, 247,
                 155, 655, 381, 517, 210])


uscae = np.array([401, 352, 604, 290, 242, 608, 376, 456, 659, 319,
                  634, 210, 594, 461, 344, 377, 354, 188, 273, 540,
                  359, 357, 546, 127, 296, 481, 436, 259, 653, 266,
                  432, 525, 514, 327, 575, 439, 234, 598, 551, 545,
                  177, 517, 125, 469, 655])

print(len(union))

58


# MNIST

In [3]:
class mnist_flattened(Dataset):
    """
    mnist_Data
    """
    def __init__(self, data):
        self.num_data = len(data)
        self.data = []

        for x, y in data:
            x = x.flatten()
        
            self.data.append((x, y))

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.num_data

train_data = visiondatasets.MNIST(root='', train=True, transform=transforms.transforms.ToTensor(), download=True)
test_data = visiondatasets.MNIST(root='', train=False, transform=transforms.transforms.ToTensor(), download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



# CompFS Models

In [4]:
class mlp(nn.Module):

    def __init__(self, indices, nhidden, nclasses):
        super(mlp, self).__init__()
        self.indices = indices
        self.fc1 = nn.Linear(len(indices), nhidden)
        self.fc2 = nn.Linear(nhidden, nclasses)

    def forward(self, x):
        x = x[:, self.indices]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
class compfs(nn.Module):

    def __init__(self, indices_list, nhidden, nclasses):
        super(compfs, self).__init__()
        self.nlearners = len(indices_list)
        self.learners = nn.ModuleList([mlp(ids, nhidden, nhidden) for ids in indices_list])
        self.fc = nn.Linear(nhidden, nclasses)

    def forward(self, x):
        out = 0
        for l in self.learners:
            out += l(x)
        out = self.fc(F.relu(out))
        return out

In [6]:
class full_model:

    def __init__(self, name, model, learning_rate):
        self.name = name
        self.model = model
        self.optimiser = torch.optim.Adam(self.model.parameters(), learning_rate)
        self.loss_ = nn.CrossEntropyLoss()
        
        self.print_epochs = 1

    def train(self, nepochs, train_data, test_data, batchsize):

        train_loader_ = torch.utils.data.DataLoader(mnist_flattened(train_data), batch_size=batchsize, shuffle=True)
        test_loader_ = torch.utils.data.DataLoader(mnist_flattened(test_data), batch_size=len(test_data), shuffle=False)
        for epoch in range(1, nepochs+1):
            
            # train for one epoch
            avg_loss = 0.0
            for x, y, in train_loader_:
                self.optimiser.zero_grad()
                output = self.model(x)

                loss = self.loss_(output, y)
                loss.backward()
                self.optimiser.step()
                avg_loss += loss.item()/len(train_loader_)
            
            # print the outcomes
            if epoch%self.print_epochs==0:
                with torch.no_grad():
                    for x, y in test_loader_:  
                        output = self.model(x)
                        preds = torch.argmax(output, dim=-1)
                        acc = 100*torch.sum(preds==y).item()//len(y)

                print('Epoch: {} Avg Loss: {:.3f} Test Acc: {:.2f}%'.format(epoch, avg_loss, acc))

    def get_individual_accuracy(self, test_data):
        ncorrect = np.zeros(10)
        ntotal = np.zeros(10)
        test_loader_ = torch.utils.data.DataLoader(mnist_flattened(test_data), batch_size=len(test_data), shuffle=False)
        with torch.no_grad():
                for x, y in test_loader_:  
                    output = self.model(x)
                    preds = torch.argmax(output, dim=-1)
                    correct = (preds==y).int()
                    acc = 100*torch.sum(preds==y).item()//len(y)

                    for c, digit in zip(correct, y):
                        ncorrect[digit] += c
                        ntotal[digit] += 1

                print('\n')
                print(ncorrect)
                print(ntotal)
                print('\n')
                print(self.name+':')
                print('Total Acc: {:.2f}%'.format(acc))
                for digit in range(10):
                    print('Digit: {} Acc: {:.2f}%'.format(digit, 100*ncorrect[digit]/ntotal[digit]))



In [7]:
nhidden = 200
nclasses = 10
learning_rate = 0.001
batchsize = 250
nepochs = 30

In [8]:
group1 = full_model('Group 1', mlp(g1, nhidden, nclasses), learning_rate)
group1.train(nepochs, train_data, test_data, batchsize)
group1.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 1.423 Test Acc: 65.00%
Epoch: 2 Avg Loss: 1.044 Test Acc: 68.00%
Epoch: 3 Avg Loss: 0.967 Test Acc: 70.00%
Epoch: 4 Avg Loss: 0.913 Test Acc: 71.00%
Epoch: 5 Avg Loss: 0.872 Test Acc: 72.00%
Epoch: 6 Avg Loss: 0.841 Test Acc: 72.00%
Epoch: 7 Avg Loss: 0.817 Test Acc: 73.00%
Epoch: 8 Avg Loss: 0.799 Test Acc: 74.00%
Epoch: 9 Avg Loss: 0.782 Test Acc: 74.00%
Epoch: 10 Avg Loss: 0.770 Test Acc: 75.00%
Epoch: 11 Avg Loss: 0.759 Test Acc: 75.00%
Epoch: 12 Avg Loss: 0.750 Test Acc: 75.00%
Epoch: 13 Avg Loss: 0.741 Test Acc: 76.00%
Epoch: 14 Avg Loss: 0.734 Test Acc: 76.00%
Epoch: 15 Avg Loss: 0.728 Test Acc: 76.00%
Epoch: 16 Avg Loss: 0.722 Test Acc: 76.00%
Epoch: 17 Avg Loss: 0.716 Test Acc: 76.00%
Epoch: 18 Avg Loss: 0.711 Test Acc: 76.00%
Epoch: 19 Avg Loss: 0.705 Test Acc: 76.00%
Epoch: 20 Avg Loss: 0.701 Test Acc: 76.00%
Epoch: 21 Avg Loss: 0.697 Test Acc: 76.00%
Epoch: 22 Avg Loss: 0.692 Test Acc: 77.00%
Epoch: 23 Avg Loss: 0.689 Test Acc: 77.00%
Epoch: 24 Avg Loss: 

In [9]:
group2 = full_model('Group 2', mlp(g2, nhidden, nclasses), learning_rate)
group2.train(nepochs, train_data, test_data, batchsize)
group2.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 1.397 Test Acc: 64.00%
Epoch: 2 Avg Loss: 1.012 Test Acc: 66.00%
Epoch: 3 Avg Loss: 0.951 Test Acc: 68.00%
Epoch: 4 Avg Loss: 0.907 Test Acc: 69.00%
Epoch: 5 Avg Loss: 0.875 Test Acc: 70.00%
Epoch: 6 Avg Loss: 0.853 Test Acc: 70.00%
Epoch: 7 Avg Loss: 0.835 Test Acc: 71.00%
Epoch: 8 Avg Loss: 0.821 Test Acc: 71.00%
Epoch: 9 Avg Loss: 0.810 Test Acc: 71.00%
Epoch: 10 Avg Loss: 0.799 Test Acc: 72.00%
Epoch: 11 Avg Loss: 0.791 Test Acc: 72.00%
Epoch: 12 Avg Loss: 0.783 Test Acc: 72.00%
Epoch: 13 Avg Loss: 0.776 Test Acc: 72.00%
Epoch: 14 Avg Loss: 0.770 Test Acc: 72.00%
Epoch: 15 Avg Loss: 0.764 Test Acc: 73.00%
Epoch: 16 Avg Loss: 0.758 Test Acc: 73.00%
Epoch: 17 Avg Loss: 0.754 Test Acc: 73.00%
Epoch: 18 Avg Loss: 0.749 Test Acc: 73.00%
Epoch: 19 Avg Loss: 0.744 Test Acc: 73.00%
Epoch: 20 Avg Loss: 0.739 Test Acc: 73.00%
Epoch: 21 Avg Loss: 0.736 Test Acc: 74.00%
Epoch: 22 Avg Loss: 0.732 Test Acc: 73.00%
Epoch: 23 Avg Loss: 0.728 Test Acc: 74.00%
Epoch: 24 Avg Loss: 

In [10]:
group3 = full_model('Group 3', mlp(g3, nhidden, nclasses), learning_rate)
group3.train(nepochs, train_data, test_data, batchsize)
group3.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 1.555 Test Acc: 58.00%
Epoch: 2 Avg Loss: 1.173 Test Acc: 62.00%
Epoch: 3 Avg Loss: 1.071 Test Acc: 65.00%
Epoch: 4 Avg Loss: 1.010 Test Acc: 67.00%
Epoch: 5 Avg Loss: 0.973 Test Acc: 68.00%
Epoch: 6 Avg Loss: 0.947 Test Acc: 68.00%
Epoch: 7 Avg Loss: 0.927 Test Acc: 69.00%
Epoch: 8 Avg Loss: 0.912 Test Acc: 69.00%
Epoch: 9 Avg Loss: 0.898 Test Acc: 70.00%
Epoch: 10 Avg Loss: 0.887 Test Acc: 70.00%
Epoch: 11 Avg Loss: 0.878 Test Acc: 70.00%
Epoch: 12 Avg Loss: 0.869 Test Acc: 70.00%
Epoch: 13 Avg Loss: 0.861 Test Acc: 70.00%
Epoch: 14 Avg Loss: 0.854 Test Acc: 70.00%
Epoch: 15 Avg Loss: 0.849 Test Acc: 71.00%
Epoch: 16 Avg Loss: 0.843 Test Acc: 71.00%
Epoch: 17 Avg Loss: 0.838 Test Acc: 71.00%
Epoch: 18 Avg Loss: 0.832 Test Acc: 71.00%
Epoch: 19 Avg Loss: 0.828 Test Acc: 71.00%
Epoch: 20 Avg Loss: 0.823 Test Acc: 71.00%
Epoch: 21 Avg Loss: 0.820 Test Acc: 71.00%
Epoch: 22 Avg Loss: 0.817 Test Acc: 71.00%
Epoch: 23 Avg Loss: 0.812 Test Acc: 71.00%
Epoch: 24 Avg Loss: 

In [11]:
group4 = full_model('Group 4', mlp(g4, nhidden, nclasses), learning_rate)
group4.train(nepochs, train_data, test_data, batchsize)
group4.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 1.294 Test Acc: 71.00%
Epoch: 2 Avg Loss: 0.893 Test Acc: 74.00%
Epoch: 3 Avg Loss: 0.824 Test Acc: 75.00%
Epoch: 4 Avg Loss: 0.779 Test Acc: 76.00%
Epoch: 5 Avg Loss: 0.748 Test Acc: 77.00%
Epoch: 6 Avg Loss: 0.722 Test Acc: 77.00%
Epoch: 7 Avg Loss: 0.703 Test Acc: 78.00%
Epoch: 8 Avg Loss: 0.687 Test Acc: 78.00%
Epoch: 9 Avg Loss: 0.674 Test Acc: 79.00%
Epoch: 10 Avg Loss: 0.663 Test Acc: 79.00%
Epoch: 11 Avg Loss: 0.653 Test Acc: 79.00%
Epoch: 12 Avg Loss: 0.645 Test Acc: 79.00%
Epoch: 13 Avg Loss: 0.637 Test Acc: 79.00%
Epoch: 14 Avg Loss: 0.631 Test Acc: 79.00%
Epoch: 15 Avg Loss: 0.625 Test Acc: 80.00%
Epoch: 16 Avg Loss: 0.620 Test Acc: 80.00%
Epoch: 17 Avg Loss: 0.615 Test Acc: 80.00%
Epoch: 18 Avg Loss: 0.611 Test Acc: 80.00%
Epoch: 19 Avg Loss: 0.607 Test Acc: 80.00%
Epoch: 20 Avg Loss: 0.603 Test Acc: 80.00%
Epoch: 21 Avg Loss: 0.600 Test Acc: 80.00%
Epoch: 22 Avg Loss: 0.597 Test Acc: 80.00%
Epoch: 23 Avg Loss: 0.594 Test Acc: 80.00%
Epoch: 24 Avg Loss: 

In [12]:
compfs1234 = full_model('CompFS', compfs([g1, g2, g3, g4], 50, nclasses), learning_rate)
compfs1234.train(nepochs, train_data, test_data, batchsize)
compfs1234.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 0.963 Test Acc: 86.00%
Epoch: 2 Avg Loss: 0.444 Test Acc: 89.00%
Epoch: 3 Avg Loss: 0.371 Test Acc: 89.00%
Epoch: 4 Avg Loss: 0.328 Test Acc: 91.00%
Epoch: 5 Avg Loss: 0.298 Test Acc: 91.00%
Epoch: 6 Avg Loss: 0.276 Test Acc: 92.00%
Epoch: 7 Avg Loss: 0.260 Test Acc: 92.00%
Epoch: 8 Avg Loss: 0.246 Test Acc: 92.00%
Epoch: 9 Avg Loss: 0.234 Test Acc: 93.00%
Epoch: 10 Avg Loss: 0.224 Test Acc: 93.00%
Epoch: 11 Avg Loss: 0.216 Test Acc: 93.00%
Epoch: 12 Avg Loss: 0.208 Test Acc: 93.00%
Epoch: 13 Avg Loss: 0.202 Test Acc: 93.00%
Epoch: 14 Avg Loss: 0.195 Test Acc: 94.00%
Epoch: 15 Avg Loss: 0.189 Test Acc: 94.00%
Epoch: 16 Avg Loss: 0.185 Test Acc: 94.00%
Epoch: 17 Avg Loss: 0.180 Test Acc: 94.00%
Epoch: 18 Avg Loss: 0.176 Test Acc: 94.00%
Epoch: 19 Avg Loss: 0.172 Test Acc: 94.00%
Epoch: 20 Avg Loss: 0.167 Test Acc: 94.00%
Epoch: 21 Avg Loss: 0.162 Test Acc: 94.00%
Epoch: 22 Avg Loss: 0.159 Test Acc: 94.00%
Epoch: 23 Avg Loss: 0.156 Test Acc: 94.00%
Epoch: 24 Avg Loss: 

In [13]:
selected_features = full_model('Selected Features', mlp(union, nhidden, nclasses), learning_rate)
selected_features.train(nepochs, train_data, test_data, batchsize)
selected_features.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 0.876 Test Acc: 85.00%
Epoch: 2 Avg Loss: 0.462 Test Acc: 88.00%
Epoch: 3 Avg Loss: 0.387 Test Acc: 89.00%
Epoch: 4 Avg Loss: 0.335 Test Acc: 90.00%
Epoch: 5 Avg Loss: 0.297 Test Acc: 91.00%
Epoch: 6 Avg Loss: 0.268 Test Acc: 92.00%
Epoch: 7 Avg Loss: 0.245 Test Acc: 93.00%
Epoch: 8 Avg Loss: 0.227 Test Acc: 93.00%
Epoch: 9 Avg Loss: 0.213 Test Acc: 93.00%
Epoch: 10 Avg Loss: 0.202 Test Acc: 94.00%
Epoch: 11 Avg Loss: 0.192 Test Acc: 94.00%
Epoch: 12 Avg Loss: 0.182 Test Acc: 94.00%
Epoch: 13 Avg Loss: 0.175 Test Acc: 94.00%
Epoch: 14 Avg Loss: 0.168 Test Acc: 94.00%
Epoch: 15 Avg Loss: 0.161 Test Acc: 94.00%
Epoch: 16 Avg Loss: 0.156 Test Acc: 94.00%
Epoch: 17 Avg Loss: 0.151 Test Acc: 94.00%
Epoch: 18 Avg Loss: 0.146 Test Acc: 95.00%
Epoch: 19 Avg Loss: 0.142 Test Acc: 95.00%
Epoch: 20 Avg Loss: 0.137 Test Acc: 95.00%
Epoch: 21 Avg Loss: 0.133 Test Acc: 95.00%
Epoch: 22 Avg Loss: 0.130 Test Acc: 95.00%
Epoch: 23 Avg Loss: 0.127 Test Acc: 95.00%
Epoch: 24 Avg Loss: 

In [16]:
modelstg = full_model('STG', mlp(stg, nhidden, nclasses), learning_rate)
modelstg.train(nepochs, train_data, test_data, batchsize)
modelstg.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 0.895 Test Acc: 85.00%
Epoch: 2 Avg Loss: 0.455 Test Acc: 88.00%
Epoch: 3 Avg Loss: 0.377 Test Acc: 89.00%
Epoch: 4 Avg Loss: 0.328 Test Acc: 90.00%
Epoch: 5 Avg Loss: 0.293 Test Acc: 91.00%
Epoch: 6 Avg Loss: 0.267 Test Acc: 92.00%
Epoch: 7 Avg Loss: 0.247 Test Acc: 92.00%
Epoch: 8 Avg Loss: 0.232 Test Acc: 93.00%
Epoch: 9 Avg Loss: 0.218 Test Acc: 93.00%
Epoch: 10 Avg Loss: 0.207 Test Acc: 93.00%
Epoch: 11 Avg Loss: 0.198 Test Acc: 93.00%
Epoch: 12 Avg Loss: 0.190 Test Acc: 93.00%
Epoch: 13 Avg Loss: 0.183 Test Acc: 94.00%
Epoch: 14 Avg Loss: 0.177 Test Acc: 93.00%
Epoch: 15 Avg Loss: 0.171 Test Acc: 94.00%
Epoch: 16 Avg Loss: 0.166 Test Acc: 94.00%
Epoch: 17 Avg Loss: 0.161 Test Acc: 94.00%
Epoch: 18 Avg Loss: 0.156 Test Acc: 94.00%
Epoch: 19 Avg Loss: 0.152 Test Acc: 94.00%
Epoch: 20 Avg Loss: 0.148 Test Acc: 94.00%
Epoch: 21 Avg Loss: 0.145 Test Acc: 94.00%
Epoch: 22 Avg Loss: 0.141 Test Acc: 94.00%
Epoch: 23 Avg Loss: 0.138 Test Acc: 94.00%
Epoch: 24 Avg Loss: 

In [17]:
modelscae = full_model('Supervised CAE', mlp(scae, nhidden, nclasses), learning_rate)
modelscae.train(nepochs, train_data, test_data, batchsize)
modelscae.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 0.915 Test Acc: 86.00%
Epoch: 2 Avg Loss: 0.451 Test Acc: 88.00%
Epoch: 3 Avg Loss: 0.385 Test Acc: 89.00%
Epoch: 4 Avg Loss: 0.338 Test Acc: 90.00%
Epoch: 5 Avg Loss: 0.303 Test Acc: 91.00%
Epoch: 6 Avg Loss: 0.277 Test Acc: 92.00%
Epoch: 7 Avg Loss: 0.255 Test Acc: 92.00%
Epoch: 8 Avg Loss: 0.239 Test Acc: 93.00%
Epoch: 9 Avg Loss: 0.224 Test Acc: 93.00%
Epoch: 10 Avg Loss: 0.211 Test Acc: 93.00%
Epoch: 11 Avg Loss: 0.201 Test Acc: 93.00%
Epoch: 12 Avg Loss: 0.192 Test Acc: 94.00%
Epoch: 13 Avg Loss: 0.184 Test Acc: 94.00%
Epoch: 14 Avg Loss: 0.177 Test Acc: 94.00%
Epoch: 15 Avg Loss: 0.171 Test Acc: 94.00%
Epoch: 16 Avg Loss: 0.165 Test Acc: 94.00%
Epoch: 17 Avg Loss: 0.160 Test Acc: 94.00%
Epoch: 18 Avg Loss: 0.155 Test Acc: 94.00%
Epoch: 19 Avg Loss: 0.151 Test Acc: 94.00%
Epoch: 20 Avg Loss: 0.147 Test Acc: 94.00%
Epoch: 21 Avg Loss: 0.143 Test Acc: 94.00%
Epoch: 22 Avg Loss: 0.140 Test Acc: 94.00%
Epoch: 23 Avg Loss: 0.137 Test Acc: 95.00%
Epoch: 24 Avg Loss: 

In [18]:
modeluscae = full_model('Unsupervised CAE', mlp(uscae, nhidden, nclasses), learning_rate)
modeluscae.train(nepochs, train_data, test_data, batchsize)
modeluscae.get_individual_accuracy(test_data)

Epoch: 1 Avg Loss: 0.933 Test Acc: 85.00%
Epoch: 2 Avg Loss: 0.460 Test Acc: 88.00%
Epoch: 3 Avg Loss: 0.396 Test Acc: 89.00%
Epoch: 4 Avg Loss: 0.350 Test Acc: 90.00%
Epoch: 5 Avg Loss: 0.313 Test Acc: 91.00%
Epoch: 6 Avg Loss: 0.284 Test Acc: 92.00%
Epoch: 7 Avg Loss: 0.261 Test Acc: 92.00%
Epoch: 8 Avg Loss: 0.242 Test Acc: 93.00%
Epoch: 9 Avg Loss: 0.227 Test Acc: 93.00%
Epoch: 10 Avg Loss: 0.214 Test Acc: 93.00%
Epoch: 11 Avg Loss: 0.203 Test Acc: 93.00%
Epoch: 12 Avg Loss: 0.194 Test Acc: 93.00%
Epoch: 13 Avg Loss: 0.186 Test Acc: 94.00%
Epoch: 14 Avg Loss: 0.178 Test Acc: 94.00%
Epoch: 15 Avg Loss: 0.172 Test Acc: 94.00%
Epoch: 16 Avg Loss: 0.166 Test Acc: 94.00%
Epoch: 17 Avg Loss: 0.161 Test Acc: 94.00%
Epoch: 18 Avg Loss: 0.156 Test Acc: 94.00%
Epoch: 19 Avg Loss: 0.151 Test Acc: 94.00%
Epoch: 20 Avg Loss: 0.147 Test Acc: 94.00%
Epoch: 21 Avg Loss: 0.143 Test Acc: 94.00%
Epoch: 22 Avg Loss: 0.139 Test Acc: 95.00%
Epoch: 23 Avg Loss: 0.136 Test Acc: 95.00%
Epoch: 24 Avg Loss: 